In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [2]:
train_df = pd.read_csv("../../../data/non-kaggle/Jeju-traffic/train.csv")
test_df = pd.read_csv("../../../data/non-kaggle/Jeju-traffic/test.csv")
data_info = pd.read_csv("../../../data/non-kaggle/Jeju-traffic/data_info.csv")
sample_submission = pd.read_csv("../../../data/non-kaggle/Jeju-traffic/sample_submission.csv")

In [3]:
print(train_df.head())

              id  base_date day_of_week  base_hour  lane_count  road_rating  \
0  TRAIN_0000000   20220623           목         17           1          106   
1  TRAIN_0000001   20220728           목         21           2          103   
2  TRAIN_0000002   20211010           일          7           2          103   
3  TRAIN_0000003   20220311           금         13           2          107   
4  TRAIN_0000004   20211005           화          8           2          103   

   road_name  multi_linked  connect_code  maximum_speed_limit  ...  road_type  \
0  지방도1112호선             0             0                 60.0  ...          3   
1   일반국도11호선             0             0                 60.0  ...          0   
2   일반국도16호선             0             0                 80.0  ...          0   
3        태평로             0             0                 50.0  ...          0   
4   일반국도12호선             0             0                 80.0  ...          0   

   start_node_name  start_latitude  st

In [4]:
print(test_df.head())

            id  base_date day_of_week  base_hour  lane_count  road_rating  \
0  TEST_000000   20220825           목         17           3          107   
1  TEST_000001   20220809           화         12           2          103   
2  TEST_000002   20220805           금          2           1          103   
3  TEST_000003   20220818           목         23           3          103   
4  TEST_000004   20220810           수         17           3          106   

  road_name  multi_linked  connect_code  maximum_speed_limit  ...  \
0       연삼로             0             0                 70.0  ...   
1  일반국도12호선             0             0                 70.0  ...   
2  일반국도16호선             0             0                 60.0  ...   
3  일반국도11호선             0             0                 70.0  ...   
4       번영로             0             0                 70.0  ...   

   height_restricted  road_type  start_node_name  start_latitude  \
0                0.0          0             산지2교      

In [5]:
print(data_info)

                      변수명           변수 설명
0                      id             아이디
1               base_date              날짜
2             day_of_week              요일
3               base_hour             시간대
4             road_in_use          도로사용여부
5              lane_count             차로수
6             road_rating            도로등급
7            multi_linked         중용구간 여부
8            connect_code          연결로 코드
9     maximum_speed_limit          최고속도제한
10      weight_restricted          통과제한하중
11      height_restricted          통과제한높이
12              road_type            도로유형
13         start_latitude        시작지점의 위도
14        start_longitude        시작지점의 경도
15  start_turn_restricted  시작 지점의 회전제한 유무
16           end_latitude        도착지점의 위도
17          end_longitude        도착지점의 경도
18    end_turn_restricted   도작지점의 회전제한 유무
19              road_name             도로명
20        start_node_name           시작지점명
21          end_node_name           도착지점명
22     vehicle_restricted         

In [6]:
train_df.describe()

,base_date,base_hour,lane_count,road_rating,multi_linked,connect_code,maximum_speed_limit,vehicle_restricted,weight_restricted,height_restricted,road_type,start_latitude,start_longitude,end_latitude,end_longitude,target
count,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4701217.0,4.701217e+06,4701217.0,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06
mean,2.021672e+07,1.192820e+01,1.836651e+00,1.049585e+02,4.762597e-04,2.660218e-01,6.125329e+01,0.0,5.618742e+03,0.0,6.152237e-01,3.338432e+01,1.265217e+02,3.338432e+01,1.265217e+02,4.278844e+01
std,4.555709e+03,6.722092e+00,6.877513e-01,1.840107e+00,2.181818e-02,5.227760e+00,1.213354e+01,0.0,1.395343e+04,0.0,1.211268e+00,1.012015e-01,1.563657e-01,1.011948e-01,1.563519e-01,1.595443e+01
min,2.021090e+07,0.000000e+00,1.000000e+00,1.030000e+02,0.000000e+00,0.000000e+00,3.000000e+01,0.0,0.000000e+00,0.0,0.000000e+00,3.324343e+01,1.261826e+02,3.324343e+01,1.261826e+02,1.000000e+00
25%,2.021110e+07,6.000000e+00,1.000000e+00,1.030000e+02,0.000000e+00,0.000000e+00,5.000000e+01,0.0,0.000000e+00,0.0,0.000000e+00,3.326422e+01,1.264232e+02,3.326422e+01,1.264232e+02,3.000000e+01
50%,2.022013e+07,1.200000e+01,2.000000e+00,1.060000e+02,0.000000e+00,0.000000e+00,6.000000e+01,0.0,0.000000e+00,0.0,0.000000e+00,3.341257e+01,1.265112e+02,3.341257e+01,1.265112e+02,4.300000e+01
75%,2.022050e+07,1.800000e+01,2.000000e+00,1.070000e+02,0.000000e+00,0.000000e+00,7.000000e+01,0.0,0.000000e+00,0.0,0.000000e+00,3.347804e+01,1.265840e+02,3.347804e+01,1.265840e+02,5.400000e+01
max,2.022073e+07,2.300000e+01,3.000000e+00,1.070000e+02,1.000000e+00,1.030000e+02,8.000000e+01,0.0,5.000000e+04,0.0,3.000000e+00,3.355608e+01,1.269309e+02,3.355608e+01,1.269309e+02,1.130000e+02


In [7]:
train_df[train_df['vehicle_restricted'] == 0]['vehicle_restricted']

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
4701212    0.0
4701213    0.0
4701214    0.0
4701215    0.0
4701216    0.0
Name: vehicle_restricted, Length: 4701217, dtype: float64

In [8]:
# vehicle_restricted 컬럼은 전부 0 이므로 필요가 없음

In [9]:
train_df['day_of_week'].unique()

array(['목', '일', '금', '화', '월', '수', '토'], dtype=object)

In [20]:
train_df.loc[train_df['day_of_week'] == '월','day_of_week'] ='1'
train_df.loc[train_df['day_of_week'] == '화','day_of_week'] ='2'
train_df.loc[train_df['day_of_week'] == '수','day_of_week'] ='3'
train_df.loc[train_df['day_of_week'] == '목','day_of_week'] ='4'
train_df.loc[train_df['day_of_week'] == '금','day_of_week'] ='5'
train_df.loc[train_df['day_of_week'] == '토','day_of_week'] ='6'
train_df.loc[train_df['day_of_week'] == '일','day_of_week'] ='7'

In [21]:
train_df['day_of_week'].unique()

array(['4', '7', '5', '2', '1', '3', '6'], dtype=object)

In [22]:
train_df.head()

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,20220623,4,17,1,106,지방도1112호선,0,0,60.0,...,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,20220728,4,21,2,103,일반국도11호선,0,0,60.0,...,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,20211010,7,7,2,103,일반국도16호선,0,0,80.0,...,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,20220311,5,13,2,107,태평로,0,0,50.0,...,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,20211005,2,8,2,103,일반국도12호선,0,0,80.0,...,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0


In [31]:
model = DecisionTreeRegressor()

In [32]:
columns = ['day_of_week','road_type','start_latitude','start_longitude','end_latitude','end_longitude','base_hour']

In [33]:
train_X, val_X, train_y, val_y = train_test_split(train_df[columns], train_df['target'])

In [34]:
model.fit(train_X, train_y)

DecisionTreeRegressor()

In [35]:
val_predictions = model.predict(val_X)

In [36]:
MAE = mean_absolute_error(val_y, val_predictions)

In [37]:
print(MAE)

3.5596772224798623


In [ ]:
# sample_submission['target'] = predictions

In [ ]:
# sample_submission

In [ ]:
# sample_submission.to_csv("../../../data/non-kaggle/Jeju-traffic/submit/Jeju_submit01.csv", index=False)